This is my modified version of `run_hw2.ipynb`.

In [26]:
%cd C:\Users\DeadScholar\Programming\CS 285 Fall 2022\CS-285_fall2022\hw2
%pdb

C:\Users\DeadScholar\Programming\CS 285 Fall 2022\CS-285_fall2022\hw2
Automatic pdb calling has been turned OFF


In [27]:
#@title test GPU availability

import torch
torch.cuda.is_available()

False

In [28]:
# plays an audio when done
from IPython.lib.display import Audio
import numpy as np

def notify(play_time_seconds=5):
    framerate = 4410

    t = np.linspace(0, play_time_seconds, framerate*play_time_seconds)
    audio_data = np.sin(2*np.pi*255*t) + np.sin(2*np.pi*240*t)
    return Audio(audio_data, rate=framerate, autoplay=True)


## Run Policy Gradients

In [30]:
#@title imports

import os
import time

from cs285.infrastructure.rl_trainer import RL_Trainer
from cs285.agents.pg_agent import PGAgent

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
--env_name Hopper-v4 --ep_len 1000 --discount 0.99 -n 300 -l 2 -s 32 -b 2000 -lr 0.001 --reward_to_go --nn_baseline --action_noise_std 0.5 --gae_lambda {lam}

In [31]:
#@title runtime arguments


my_env_name = "Hopper-v4"
exp_name_dict = {
    "InvertedPendulum-v4": "ivp",
    "CartPole-v0": "cp",
    "LunarLanderContinuous-v2": "luna",
    "Hopper-v4": "Hopper"
}
lam = 0.95
action_noise_std = 0.5

In [32]:
class Args:

  def __getitem__(self, key):
    return getattr(self, key)

  def __setitem__(self, key, val):
    setattr(self, key, val)

  def __contains__(self, key):
    return hasattr(self, key)

  env_name = my_env_name #@param
  exp_name = 'q1_' + exp_name_dict[env_name] #@param

  #@markdown main parameters of interest
  n_iter = 100 #@param {type: "integer"}

  ## PDF will tell you how to set ep_len
  ## and discount for each environment
  ep_len = 1000 #@param {type: "integer"}
  discount = 0.99 #@param {type: "number"}

  reward_to_go = True #@param {type: "boolean"}
  nn_baseline = True #@param {type: "boolean"}
  gae_lambda = lam #@param {type: "float"}
  dont_standardize_advantages = False #@param {type: "boolean"}

  #@markdown batches and steps
  batch_size = 40000 #@param {type: "integer"}
  eval_batch_size = 400 #@param {type: "integer"}

  num_agent_train_steps_per_iter = 1 #@param {type: "integer"}
  learning_rate =  5e-3 #@param {type: "number"}

  #@markdown MLP parameters
  n_layers = 2 #@param {type: "integer"}
  size = 64 #@param {type: "integer"}

  #@markdown system
  save_params = False #@param {type: "boolean"}
  no_gpu = False #@param {type: "boolean"}
  which_gpu = 0 #@param {type: "integer"}
  seed = 1 #@param {type: "integer"}
    
  action_noise_std =  #@param {type: "float"}

  #@markdown logging
  ## default is to not log video so
  ## that logs are small enough to be
  ## uploaded to gradscope
  video_log_freq =  10#@param {type: "integer"}
  scalar_log_freq =  1#@param {type: "integer"}


args = Args()

## ensure compatibility with hw1 code
args['train_batch_size'] = args['batch_size']

if args['video_log_freq'] > 0:
  import warnings
  warnings.warn(
      '''\nLogging videos will make eventfiles too'''
      '''\nlarge for the autograder. Set video_log_freq = -1'''
      '''\nfor the runs you intend to submit.''')

In [33]:
#@title create directory for logging

data_path = '''./data'''

if not (os.path.exists(data_path)):
    os.makedirs(data_path)

logdir = args.exp_name + '_' + args.env_name + '_' + time.strftime("%d-%m-%Y_%H-%M-%S")
logdir = os.path.join(data_path, logdir)
args['logdir'] = logdir
if not(os.path.exists(logdir)):
    os.makedirs(logdir)

In [34]:
## define policy gradient trainer

class PG_Trainer(object):

    def __init__(self, params):

        #####################
        ## SET AGENT PARAMS
        #####################

        computation_graph_args = {
            'n_layers': params['n_layers'],
            'size': params['size'],
            'learning_rate': params['learning_rate'],
            }

        estimate_advantage_args = {
            'gamma': params['discount'],
            'standardize_advantages': not(params['dont_standardize_advantages']),
            'reward_to_go': params['reward_to_go'],
            'nn_baseline': params['nn_baseline'],
            'gae_lambda': params['gae_lambda'],
        }

        train_args = {
            'num_agent_train_steps_per_iter': params['num_agent_train_steps_per_iter'],
        }

        agent_params = {**computation_graph_args, **estimate_advantage_args, **train_args}

        self.params = params
        self.params['agent_class'] = PGAgent
        self.params['agent_params'] = agent_params
        self.params['batch_size_initial'] = self.params['batch_size']

        ################
        ## RL TRAINER
        ################

        self.rl_trainer = RL_Trainer(self.params)

    def run_training_loop(self):

        self.rl_trainer.run_training_loop(
            self.params['n_iter'],
            collect_policy = self.rl_trainer.agent.actor,
            eval_policy = self.rl_trainer.agent.actor,
            )

In [35]:
%pdb
print(args.logdir)
trainer = PG_Trainer(args)
trainer.run_training_loop()

Automatic pdb calling has been turned ON
./data\q1_Hopper_Hopper-v4_24-09-2022_12-38-28
########################
logging outputs to  ./data\q1_Hopper_Hopper-v4_24-09-2022_12-38-28
########################
GPU not detected. Defaulting to CPU.


********** Iteration 0 ************



Training agent using sampled data from replay buffer...


UnboundLocalError: local variable 'advantages' referenced before assignment

> c:\users\deadscholar\programming\cs 285 fall 2022\cs-285_fall2022\hw2\cs285\agents\pg_agent.py(138)estimate_advantage()
    136                 for i in range(len(rews_list)):
    137                     episode_len = len(rews_list[i])
--> 138                     advantages_episode = advantages[count:count+episode_len]
    139                     values_episode = values[count:count+episode_len]
    140 



ipdb>  advantages


*** NameError: name 'advantages' is not defined


ipdb>  ll


    101     def estimate_advantage(self, obs: np.ndarray, rews_list: List[np.ndarray], q_values: np.ndarray, terminals: np.ndarray) -> np.ndarray:
    102 
    103         """
    104             Computes advantages by one of the following methods:
    105                 - A_{GAE}^π(s_t, a_t)
    106                 - A^π(s_t, a_t) = Q^π(s_t, a_t) - V_φ(s_t), where V is computed by a "baseline" neural network
    107                 - A^π(s_t, a_t) = Q^π(s_t, a_t)
    108 
    109             OUTPUT. array of estimated advantage at each step.
    110                 A^π(s_t, a_t)   ∀ t = 0, ..., T-1
    111         """ 
    112 
    113         # Estimate the advantage A = Q - V, where V is computed by the "baseline" neural network.
    114         if self.nn_baseline:
    115             values_normalized = self.actor.run_baseline_prediction(obs)
    116             ## ensure that the value predictions and q_values have the same dimensionality
    117             ## to prevent silent

ipdb>  q


In [19]:
# #@markdown You can visualize your runs with tensorboard from within the notebook

# ## requires tensorflow==2.3.0
# %load_ext tensorboard
# %tensorboard --logdir /content/cs285_f2022/homework_fall2022/hw2/data